In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F

batch_size = 32
block_size = 8
max_iters = 3000
eval_interval = 300
learning_rate = 1e-2
device = "cuda" if torch.cuda.is_available() else "cpu"
eval_iters = 200

torch.manual_seed(1337)

with open("input.txt", "r", encoding="utf-8") as f:
    text = f.read()

chars = sorted(list(set(text)))
vocab_size = len(chars)

stoi = {ch: i for i, ch in enumerate(chars)}
itos = {i: ch for i, ch in enumerate(chars)}

encode = lambda s: [stoi[i] for i in s]
decode = lambda l: "".join([itos[i] for i in l])

data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == "train" else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    X = torch.stack([data[i:i + block_size] for i in ix])
    y = torch.stack([data[i + 1:i + block_size + 1] for i in ix])
    X, y = X.to(device), y.to(device)
    return X, y

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ["train", "val"]:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, y = get_batch(split)
            logits, loss = model(X, y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

class BigramLanguageModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
    
    def forward(self, idx, targets=None):
        logits = self.token_embedding_table(idx)
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B * T, C)
            targets = targets.view(B * T)
            loss = F.cross_entropy(logits, targets)
        return logits, loss

    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, loss = self(idx)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)
            idx_next = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx, idx_next), dim=1)
        return idx
    
if __name__ == "__main__":
    model = BigramLanguageModel()
    m = model.to(device)
    optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
    for iter in range(max_iters):
        if iter % eval_interval == 0:
            losses = estimate_loss()
            print(f"Step: {iter}, Loss: {losses}")
        X, y = get_batch("train")
        logits, loss = model(X, y)
        optimizer.zero_grad(set_to_none=True)
        loss.backward()
        optimizer.step()

    context = torch.zeros((1, 1), dtype=torch.long, device=device)
    print(decode(m.generate(context, max_new_tokens=500)[0].tolist()))

Step: 0, Loss: {'train': tensor(4.7305), 'val': tensor(4.7241)}
Step: 300, Loss: {'train': tensor(2.8110), 'val': tensor(2.8249)}
Step: 600, Loss: {'train': tensor(2.5434), 'val': tensor(2.5682)}
Step: 900, Loss: {'train': tensor(2.4932), 'val': tensor(2.5088)}
Step: 1200, Loss: {'train': tensor(2.4863), 'val': tensor(2.5035)}
Step: 1500, Loss: {'train': tensor(2.4665), 'val': tensor(2.4921)}
Step: 1800, Loss: {'train': tensor(2.4683), 'val': tensor(2.4936)}
Step: 2100, Loss: {'train': tensor(2.4696), 'val': tensor(2.4846)}
Step: 2400, Loss: {'train': tensor(2.4638), 'val': tensor(2.4879)}
Step: 2700, Loss: {'train': tensor(2.4738), 'val': tensor(2.4911)}



CEThik brid owindakis b, bth

HAPet bobe d e.
S:
O:3 my d?
LUCous:
Wanthar u qur, t.
War dXENDoate awice my.

Hastarom oroup
Yowhthetof isth ble mil ndill, ath iree sengmin lat Heriliovets, and Win nghir.
Swanousel lind me l.
HAshe ce hiry:
Supr aisspllw y.
Hentofu n Boopetelaves
MPOLI s, d mothakleo Windo whth eisbyo the m dourive

In [ ]:
torch.save(model, "./bigram_language_model.pt")

In [ ]:
model = torch.load("./bigram_language_model.pt")
model.eval()

BigramLanguageModel(
  (token_embedding_table): Embedding(65, 65)
)

In [ ]:
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(model.generate(context, max_new_tokens=500)[0].tolist()))


MPrrtano iru forealoiroret HEnk;
CKES:MO tck in, d cer t ftanofallon bay ho s, andemen, meseveminds s; te worimyoin ie-s ofit thiWhou wowhedichea blare aned hy senInirmy a theint d polofind yon be ke imyol menatoulinor woeing brwimapise. hawe e wo IICis;
ST:
GRDo y'silH:
FFortaperulllalop!
Antheolirconourjura d f t wothrmu th?
Whaling mas, h n t il c; d junlotir Whth'dlll my ay meras be:
HERGSe ndonbust RIUCE:


Anveaban; IOn hequ thitan nu tonm TESCorte y, d.
Sl!
HAUSHAny IE:
ARERDO:
BRolathon



In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F

batch_size = 64
block_size = 256
max_iters = 5000
eval_interval = 500
learning_rate = 3e-4
device = "cuda" if torch.cuda.is_available() else "cpu"
eval_iters = 200
n_embds = 384
n_heads = 6
head_size = n_embds // n_heads
n_layers = 6
dropout = 0.2

torch.manual_seed(1337)

with open("input.txt", "r", encoding="utf-8") as f:
    text = f.read()

chars = sorted(list(set(text)))
vocab_size = len(chars)

stoi = {ch: i for i, ch in enumerate(chars)}
itos = {i: ch for i, ch in enumerate(chars)}

encode = lambda s: [stoi[i] for i in s]
decode = lambda l: "".join([itos[i] for i in l])

data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == "train" else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    X = torch.stack([data[i:i + block_size] for i in ix])
    y = torch.stack([data[i + 1:i + block_size + 1] for i in ix])
    X, y = X.to(device), y.to(device)
    return X, y

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ["train", "val"]:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, y = get_batch(split)
            logits, loss = model(X, y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

class Head(nn.Module):
    def __init__(self):
        super().__init__()
        self.key = nn.Linear(n_embds, head_size, bias=False)
        self.query = nn.Linear(n_embds, head_size, bias=False)
        self.value = nn.Linear(n_embds, head_size, bias=False)
        self.register_buffer("tril", torch.tril(torch.ones(block_size, block_size)))
        self.dropout = nn.Dropout(dropout)
    
    def forward(self, X):
        B, T, C = X.shape
        k = self.key(X)
        q = self.query(X)
        wei = q @ k.transpose(-2, -1) * k.shape[-1] ** -0.5
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float("-inf"))
        wei = F.softmax(wei, dim=-1)
        wei = self.dropout(wei)
        v = self.value(X)
        out = wei @ v
        return out

class MultiHeadAttention(nn.Module):
    def __init__(self):
        super().__init__()
        self.heads = nn.ModuleList([Head() for _ in range(n_heads)])
        self.proj = nn.Linear(head_size * n_heads, n_embds)
        self.dropout = nn.Dropout(dropout)
    
    def forward(self, X):
        out = torch.cat([h(X) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedForward(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embds, n_embds * 4),
            nn.ReLU(),
            nn.Linear(n_embds * 4, n_embds),
            nn.Dropout(dropout), 
        )
    
    def forward(self, X):
        return self.net(X)

class Block(nn.Module):
    def __init__(self):
        super().__init__()
        self.sa = MultiHeadAttention()
        self.ffwd = FeedForward()
        self.ln1 = nn.LayerNorm(n_embds)
        self.ln2 = nn.LayerNorm(n_embds)
    
    def forward(self, X):
        X = X + self.sa(self.ln1(X))
        X = X + self.ffwd(self.ln2(X))
        return X

class GPTLanguageModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embds)
        self.position_embedding_table = nn.Embedding(block_size, n_embds)
        self.blocks = nn.Sequential(*[Block() for _ in range(n_layers)])
        self.ln_f = nn.LayerNorm(n_embds)
        self.lm_head = nn.Linear(n_embds, vocab_size)
        self.apply(self._init_weights)
    
    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
    
    def forward(self, idx, targets=None):
        B, T = idx.shape
        tok_emb = self.token_embedding_table(idx)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device))
        X = tok_emb + pos_emb
        X = self.blocks(X)
        X = self.ln_f(X)
        logits = self.lm_head(X)
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B * T, C)
            targets = targets.view(B * T)
            loss = F.cross_entropy(logits, targets)
        return logits, loss

    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            idx_cond = idx[:, -block_size:]
            logits, loss = self(idx_cond)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)
            idx_next = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx, idx_next), dim=1)
        return idx
    
if __name__ == "__main__":
    model = GPTLanguageModel()
    m = model.to(device)
    optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
    for iter in range(max_iters):
        if iter % eval_interval == 0:
            losses = estimate_loss()
            print(f"Step: {iter}, Loss: {losses}")
        X, y = get_batch("train")
        logits, loss = model(X, y)
        optimizer.zero_grad(set_to_none=True)
        loss.backward()
        optimizer.step()

    context = torch.zeros((1, 1), dtype=torch.long, device=device)
    print(decode(m.generate(context, max_new_tokens=500)[0].tolist()))

Step: 0, Loss: {'train': tensor(4.2221), 'val': tensor(4.2306)}
Step: 500, Loss: {'train': tensor(1.7599), 'val': tensor(1.9162)}
Step: 1000, Loss: {'train': tensor(1.3933), 'val': tensor(1.6023)}
Step: 1500, Loss: {'train': tensor(1.2685), 'val': tensor(1.5291)}
Step: 2000, Loss: {'train': tensor(1.1868), 'val': tensor(1.4955)}
Step: 2500, Loss: {'train': tensor(1.1219), 'val': tensor(1.4860)}
Step: 3000, Loss: {'train': tensor(1.0737), 'val': tensor(1.4861)}
Step: 3500, Loss: {'train': tensor(1.0215), 'val': tensor(1.5081)}
Step: 4000, Loss: {'train': tensor(0.9648), 'val': tensor(1.5082)}
Step: 4500, Loss: {'train': tensor(0.9082), 'val': tensor(1.5378)}

But with prison chariteds entremitted him
if well, Camillia in's popularison.

MISTRESS OVERDONE:
Your first shall want too heavy that was the fearful venture.

MISTRESSOP OF S:
Master may shake her recovery heaven his right;
Much trod many indeed soon stand here hours as are steep:
More citizens afterward than this does and discor

In [ ]:
torch.save(model, "./gpt_language_model.pt")

In [ ]:
model = torch.load("./gpt_language_model.pt")
model.eval()

GPTLanguageModel(
  (token_embedding_table): Embedding(65, 384)
  (position_embedding_table): Embedding(256, 384)
  (blocks): Sequential(
    (0): Block(
      (sa): MultiHeadAttention(
        (heads): ModuleList(
          (0-5): 6 x Head(
            (key): Linear(in_features=384, out_features=64, bias=False)
            (query): Linear(in_features=384, out_features=64, bias=False)
            (value): Linear(in_features=384, out_features=64, bias=False)
            (dropout): Dropout(p=0.2, inplace=False)
          )
        )
        (proj): Linear(in_features=384, out_features=384, bias=True)
        (dropout): Dropout(p=0.2, inplace=False)
      )
      (ffwd): FeedForward(
        (net): Sequential(
          (0): Linear(in_features=384, out_features=1536, bias=True)
          (1): ReLU()
          (2): Linear(in_features=1536, out_features=384, bias=True)
          (3): Dropout(p=0.2, inplace=False)
        )
      )
      (ln1): LayerNorm((384,), eps=1e-05, elementwise_affine

In [ ]:
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(model.generate(context, max_new_tokens=500)[0].tolist()))


My lords, there's reason's face to answer me,
For something what the disobed sun
Doth death once and our children, with triumphants
Will perceive a courtier envious to visit
Tirench of the hand wrongs: seldom and your wrath,
With this blood of slaves dreadful majesty,
That it contempt shall from this hard-happiness
Action water of a firmer's face,
Trempted business on unthroat the field's death.
Here in this; our aery of us sweet souls
Are the waste in the field-shining toward with child.
Ah, pe
